In [4]:
import os
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords

# Baixa os stopwords em português
nltk.download('stopwords')

# Carrega o CSV do diretório data
csv_path = os.path.join('..', 'data', 'TweetsWithTheme.csv')
df = pd.read_csv(csv_path)

stop_words = set(stopwords.words('portuguese'))

def clean_tweet(text):
    if not isinstance(text, str):
        return ''
    text = text.lower()
    text = re.sub(r'http\S+|www\S+', '', text)  # remove URLs
    text = re.sub(r'@\w+|#\w+', '', text)       # remove mentions/hashtags
    text = re.sub(r'[^a-zà-ú\s]', '', text)     # mantém apenas letras
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Assumindo que a coluna de texto se chama 'tweet_text' (ajuste conforme necessário)
if 'tweet_text' in df.columns:
    df['clean_text'] = df['tweet_text'].apply(clean_tweet)
else:
    # Se a coluna tiver outro nome, vamos descobrir e usar
    print("Colunas disponíveis:", df.columns.tolist())
    text_column = [col for col in df.columns if 'text' in col.lower() or 'tweet' in col.lower()]
    if text_column:
        df['clean_text'] = df[text_column[0]].apply(clean_tweet)
    else:
        print("Nenhuma coluna de texto encontrada!")

df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thiag\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,tweet_text,tweet_date,sentiment,query_used,clean_text
0,1026606627041026050,ela vendo a família :( FUGIDINHA VERA LÚCIA,Mon Aug 06 23:11:07 +0000 2018,Negativo,vera lucia :(,vendo família fugidinha vera lúcia
1,1026266117646561281,Eu odeio o fato que muito provavelmente meu vo...,Mon Aug 06 00:38:04 +0000 2018,Negativo,ciro :(,odeio fato provavelmente voto vai ter ir pro c...
2,1025889796097564672,"Ciro ""mita"" tanto quanto aquele outro, não que...",Sat Aug 04 23:42:42 +0000 2018,Negativo,ciro :(,ciro mita tanto quanto outro quero obrigada vo...
3,1026960047937646598,Raiva do Bolsonaro agora :),Tue Aug 07 22:35:30 +0000 2018,Positivo,bolsonaro :),raiva bolsonaro agora
4,1026447105932623872,Reflexos da alienação petista comandada por Lu...,Mon Aug 06 12:37:15 +0000 2018,Negativo,lula :(,reflexos alienação petista comandada lula país...
